Hopefully last setup necessary for preparing the environment for Inverse RL procedure on conversational AI. In this version we use pre-trained word embeddings and combine them with the raw states to provide a clear flow of logic as a part of the convo.

In [28]:
import torch
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import numpy as np
from io import open
import itertools
import math
import matplotlib.pyplot as plt
%matplotlib inline


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")



/scratch/nsk367/anaconda3/envs/irl/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:

d = torch.load('../apps/dat/preprocess/padded_vectorized_states.pt')#, map_location=l$
raw = torch.load('../apps/dat/preprocess/raw_states.pt')#, map_location=lambda storag$



In [6]:
for index, vects in d.items():
    input_state, next_state = vects[0], vects[1]
    raw_input_state, raw_next_state = list(raw.keys())[index], raw[list(raw.keys())[index]]
    print(raw_input_state,"\n", vects[0])
    print(raw_next_state,"\n", vects[1])
    break
                                                                        
                                                        



Not the hacking and gagging and spitting part . Please .   
 tensor([[-0.0498,  0.0270, -0.3879,  ..., -0.3036,  0.2707,  0.1805],
        [ 0.2720, -0.0620, -0.1884,  ...,  0.1302, -0.1832,  0.1323],
        [-0.8311, -0.0013,  0.1606,  ...,  0.7167, -1.0708,  1.1857],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
Okay ... then how 'bout we try out some <norp> cuisine . <date> ? <time> ?  
 tensor([[-0.3453, -0.0665, -0.5133,  ..., -0.1900,  0.0508,  0.0804],
        [ 0.0018,  0.2487, -0.0855,  ...,  0.0145,  0.0465,  0.1413],
        [ 0.1472,  0.0786, -0.3404,  ...,  0.0515,  0.1082,  0.1751],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0

In [ ]:
    input_state, next_state = vects[0], vects[1]
    raw_input_state, raw_next_state = list(raw.keys())[index], raw[list(raw.keys())[index]]
    print(raw_input_state,"\n", vects[0])
    print(raw_next_state,"\n", vects[1])
    break

In [17]:
i = 0
input_state = d[i][0]
response = d[i][1]

raw_input_state, raw_response = list(raw.keys())[i], raw[list(raw.keys())[i]]


{0: [tensor([[-0.0498,  0.0270, -0.3879,  ..., -0.3036,  0.2707,  0.1805],
          [ 0.2720, -0.0620, -0.1884,  ...,  0.1302, -0.1832,  0.1323],
          [-0.8311, -0.0013,  0.1606,  ...,  0.7167, -1.0708,  1.1857],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([[-0.3453, -0.0665, -0.5133,  ..., -0.1900,  0.0508,  0.0804],
          [ 0.0018,  0.2487, -0.0855,  ...,  0.0145,  0.0465,  0.1413],
          [ 0.1472,  0.0786, -0.3404,  ...,  0.0515,  0.1082,  0.1751],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])],
 1: [tensor([[-0.1249,  0.0692, -0.3136,  ..., -0.4034,  0.1776,  0.0892],
          [-0.0828,  0.

Now put the environment to the test

In [42]:
class DialogEnvironment(object):
    """
    
    Gym environment for dialog.
    
    """
    def __init__(self, mode='train'):
        

        self.conversations = d
        self.raw_conversations = raw
        
    
       # if mode == 'train':
         #   self.conversations = self.conversations[:-10]
      #  elif mode == 'test':
           # self.conversations = self.conversations[-10]
            
        
        self.conversations_visited = []
        
    def clear(self):
        self.conversations_visited = [] #
    def current_state(self):
        return i  # i for current conversation index, j for current word (these should be odd? )
    
    def reset(self):
        # pick a random i in length of convos. If in convos visited, skip
        while True:
            try:
                self.i = random.randint(a=0,b=len(self.conversations))


                self.conversations_visited.append(self.i)
                self.conversation = self.conversations[self.i]
                break
            except:
                pass


        state = self.conversation[0]
        expert_action = self.conversation[0]
        
        raw_state = list(self.raw_conversations.keys())[self.i], 
        
        raw_expert_action = self.raw_conversations[list(self.raw_conversations.keys())[self.i]]

        return state, expert_action, raw_state, raw_expert_action
    
    def step(self,action):
        done = True

        return done


In [55]:
env = DialogEnvironment()

In [63]:
state, expert_action, raw_state, raw_expert_action = env.reset()
print(raw_state,raw_expert_action)

("Aw , I do . But ' seems to me us colored folks do a whole lot of church - goin ' . It 's the whites that need it real bad .  ",) I think if you tried livin ' in a town like this , instead of running free and easy , you 'd soon change your tune . 


In [62]:
print(raw_state,raw_expert_action)

("Okay .  Boy , <person> 's as slow as a snail , is n't he ?  ",) No , he 's more of a dinosaur . <person> 's not a dummy , though . He 's juggling alot of balls on this one . 


In [59]:
state.shape

torch.Size([60, 300])

In [60]:
expert_action.shape

torch.Size([60, 300])